In [4]:
import h2o
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

In [7]:
h2o.init(nthreads = 2, max_mem_size = 4)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 14.0.2+12-46, mixed mode, sharing)
  Starting server from C:\Users\Diego\anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\Diego\AppData\Local\Temp\tmplv7es1ee
  JVM stdout: C:\Users\Diego\AppData\Local\Temp\tmplv7es1ee\h2o_Diego_started_from_python.out
  JVM stderr: C:\Users\Diego\AppData\Local\Temp\tmplv7es1ee\h2o_Diego_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,America/Los_Angeles
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.1.1
H2O_cluster_version_age:,2 days
H2O_cluster_name:,H2O_from_python_Diego_oj4mrc
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,4 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"accepting new members, healthy"


In [9]:
df = h2o.import_file("loan.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [10]:
df.describe

loan_amnt,term,int_rate,emp_length,home_ownership,annual_inc,purpose,addr_state,dti,delinq_2yrs,revol_util,total_acc,bad_loan,longest_credit_length,verification_status
5000,36 months,10.65,10,RENT,24000,credit_card,AZ,27.65,0,83.7,9,0,26,verified
2500,60 months,15.27,0,RENT,30000,car,GA,1,0,9.4,4,1,12,verified
2400,36 months,15.96,10,RENT,12252,small_business,IL,8.72,0,98.5,10,0,10,not verified
10000,36 months,13.49,10,RENT,49200,other,CA,20,0,21,37,0,15,verified
5000,36 months,7.9,3,RENT,36000,wedding,AZ,11.2,0,28.3,12,0,7,verified
3000,36 months,18.64,9,RENT,48000,car,CA,5.35,0,87.5,4,0,4,verified
5600,60 months,21.28,4,OWN,40000,small_business,CA,5.55,0,32.6,13,1,7,verified
5375,60 months,12.69,0,RENT,15000,other,TX,18.08,0,36.5,3,1,7,verified
6500,60 months,14.65,5,OWN,72000,debt_consolidation,AZ,16.12,0,20.6,23,0,13,not verified
12000,36 months,12.69,10,OWN,75000,debt_consolidation,CA,10.78,0,67.1,34,0,22,verified


<bound method H2OFrame.describe of >

In [11]:
df.shape

(163987, 15)

In [12]:
split = df.split_frame(ratios = [0.7, 0.15])
train, validate, test = split

In [14]:
train.shape

(114758, 15)

In [15]:
validate.shape

(24761, 15)

In [16]:
test.shape

(24468, 15)

In [17]:
df.columns

['loan_amnt',
 'term',
 'int_rate',
 'emp_length',
 'home_ownership',
 'annual_inc',
 'purpose',
 'addr_state',
 'dti',
 'delinq_2yrs',
 'revol_util',
 'total_acc',
 'bad_loan',
 'longest_credit_length',
 'verification_status']

In [26]:
x = ['loan_amnt', 'term', 'int_rate', 'emp_length', 'home_ownership', 'annual_inc', 'purpose', 'addr_state', 'dti', 'delinq_2yrs', 'revol_util', 'total_acc', 'bad_loan', 'longest_credit_length']
y = 'verification_status'

In [27]:
glm = H2OGeneralizedLinearEstimator(family = 'binomial', alpha = 0.15)

In [28]:
glm.train(x = x, y = y, training_frame = train)

glm Model Build progress: |███████████████████████████████████████████████| 100%


In [29]:
glm.model_performance(test)


ModelMetricsBinomialGLM: glm
** Reported on test data. **

MSE: 0.1905413469248778
RMSE: 0.43651042017903513
LogLoss: 0.5513902524985129
Null degrees of freedom: 24467
Residual degrees of freedom: 24432
Null deviance: 32124.261690550815
Residual deviance: 26982.83339626723
AIC: 27054.83339626723
AUC: 0.7494982602626198
AUCPR: 0.8560782820598072
Gini: 0.4989965205252396

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3309150525654554:


,,not verified,verified,Error,Rate
0,not verified,649.0,8288.0,0.9274,(8288.0/8937.0)
1,verified,378.0,15153.0,0.0243,(378.0/15531.0)
2,Total,1027.0,23441.0,0.3542,(8666.0/24468.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.330915,0.777635,358.0
1,max f2,0.201631,0.896833,397.0
2,max f0point5,0.661315,0.771724,171.0
3,max accuracy,0.542434,0.684895,241.0
4,max precision,0.994079,1.000000,0.0
5,max recall,0.201631,1.000000,397.0
6,max specificity,0.994079,1.000000,0.0
7,max absolute_mcc,0.728950,0.393054,136.0
8,max min_per_class_accuracy,0.584052,0.681996,215.0
9,max mean_per_class_accuracy,0.670872,0.695692,166.0



Gains/Lift Table: Avg response rate: 63.47 %, avg score: 63.91 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.010013,0.985869,1.575430,1.575430,1.000000,0.988668,1.000000,0.988668,0.015775,0.015775,57.542979,57.542979,0.015775
1,2,0.020026,0.980807,1.556139,1.565784,0.987755,0.983262,0.993878,0.985965,0.015582,0.031357,55.613881,56.578430,0.031021
2,3,0.030039,0.975309,1.549708,1.560426,0.983673,0.977982,0.990476,0.983304,0.015517,0.046874,54.970848,56.042569,0.046091
3,4,0.040011,0.969033,1.549603,1.557728,0.983607,0.972371,0.988764,0.980579,0.015453,0.062327,54.960307,55.772833,0.061096
4,5,0.050025,0.963565,1.530418,1.552262,0.971429,0.966126,0.985294,0.977686,0.015324,0.077651,53.041751,55.226170,0.075637
5,6,0.100008,0.938540,1.543226,1.547745,0.979558,0.951178,0.982427,0.964437,0.077136,0.154787,54.322558,54.774549,0.149976
6,7,0.150033,0.908957,1.489193,1.528223,0.945261,0.924079,0.970035,0.950981,0.074496,0.229283,48.919302,52.822268,0.216975
7,8,0.200016,0.870022,1.438884,1.505897,0.913328,0.890329,0.955864,0.935824,0.071921,0.301204,43.888395,50.589713,0.277035
8,9,0.300025,0.765889,1.321764,1.444520,0.838987,0.817934,0.916905,0.896528,0.132187,0.433391,32.176434,44.451953,0.365136
9,10,0.399992,0.681645,1.082705,1.354094,0.687244,0.721989,0.859508,0.852906,0.108235,0.541626,8.270543,35.409373,0.387772


This is the simple regression model

In [31]:
from h2o.grid.grid_search import H2OGridSearch

Define the search space

In [ ]:
param = {'alpha': [x * 0.01 for x in range(0,11)]}

Initialize the grid-search instance

In [37]:
h2o_grid = H2OGridSearch(model = H2OGeneralizedLinearEstimator(family = 'binomial'), hyper_params = param, search_criteria = {'strategy': 'Cartesian'}, grid_id = 'glm_grid1')

Train the grid

In [38]:
h2o_grid.train(x = x, y = y, training_frame = train, validation_frame = validate, lambda_search = True)

glm Grid Build progress: |████████████████████████████████████████████████| 100%


Get the best hyperparemeters from the model

In [39]:
cartesian_grid_perf = h2o_grid.get_grid(sort_by = 'auc', decreasing = True)
cartesian_grid_perf

alpha           model_ids                 auc
0      [0.0]   glm_grid1_model_1  0.7493283308247519
1     [0.05]   glm_grid1_model_6  0.7492848270058606
2     [0.04]   glm_grid1_model_5  0.7492490491207315
3     [0.02]   glm_grid1_model_3  0.7492184489171142
4     [0.03]   glm_grid1_model_4  0.7492184312699379
5     [0.01]   glm_grid1_model_2  0.7492143335956127
6      [0.1]  glm_grid1_model_11  0.7492019382190148
7     [0.09]  glm_grid1_model_10  0.7491787321822461
8     [0.08]   glm_grid1_model_9  0.7491290059690091
9     [0.07]   glm_grid1_model_8  0.7491184776636584
10    [0.06]   glm_grid1_model_7  0.7491078540635561


Now we want the best model so we will just grab the top one

In [40]:
best_cart_model = cartesian_grid_perf.models[0]

This is the random search part

In [42]:
params = {'alpha': [x * 0.01 for x in range(0,99)]}

In [43]:
# this will make sure that 30 models have been checked
search_criteria = {"strategy": "RandomDiscrete", "max_models": 30}

In [50]:
h2o_grid2 = H2OGridSearch(model = H2OGeneralizedLinearEstimator(family = 'binomial'), hyper_params = params, search_criteria = search_criteria, grid_id = 'glm_grid2')

In [51]:
h2o_grid2(x = x, y = y, training_frame = train, validaiton_frame = validate, lambda_search = True)

TypeError: 'H2OGridSearch' object is not callable